# 3. Table view

In this tutorial we'll look at how to use **QTableView** from PyQt6, including how to model your data, format values for display and add conditional formatting.

`QTableView` is a Qt view widget which presents data in a spreadsheet-like table view. Like all widgets in the Model View Architecture, this uses a separate model to provide data and presentation information to the view. Data in the model can be updated as required, and the view notified of these changes to redraw/display the changes. By customising the model it is possible to have a huge amount of control over how the data is presented.

## 3.1 First Table view model

Below code shows an example of the table view model application with some dummy data. This is a basic application structure:
1. create a custom table model by subclassing the `QAbstractTableModel`, must implement
   - data():
   - rowCount():
   - columnCount():
2.

In [1]:
import sys
from PyQt6 import QtCore, QtWidgets
from PyQt6.QtCore import Qt
# model class subclass the QAbstractTableModel
# it has a 2D data structure with rows and columns
class TableModel(QtCore.QAbstractTableModel):
    """

    """
    def __init__(self, data):
        super(TableModel, self).__init__()
        self._data = data

    # return the data which view requires
    def data(self, index, role):
        # in this example, we only treat DisplayRole
        if role == Qt.ItemDataRole.DisplayRole:
            # See below for the nested-list data structure.
            # .row() indexes into the outer list,
            # .column() indexes into the sub-list
            return self._data[index.row()][index.column()]

    # return the row count of current data store
    def rowCount(self, index):
        # The length of the outer list.
        return len(self._data)

    def columnCount(self, index):
        # The following takes the first sub-list, and returns
        # the length (only works if all rows are an equal length)
        return len(self._data[0])


class MainWindow(QtWidgets.QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Table view with custom model")
        self.setGeometry(100,100,800,600)

        # create a table view
        self.table = QtWidgets.QTableView()

        # use nested list as a 2-dimensional data store
        data = [
          [4, 9, 2],
          [1, 0, 0],
          [3, 5, 0],
          [3, 3, 2],
          [7, 8, 9],
        ]

        self.model = TableModel(data)
        self.table.setModel(self.model)

        self.setCentralWidget(self.table)


app=QtWidgets.QApplication(sys.argv)
window=MainWindow()
window.show()
app.exec()

qt.qpa.xcb: X server does not support XInput 2


0

## 3.2 Formatting numbers and dates

The view expect the data returned by the model for display is string type. For int, and float values, the default string representation is enough. But for complex python types such as date, the default format will not work. You must format them to string yourself.

Below is a simple custom formatter which looks up the values in our data table, and displays them in a number of different ways depending on the Python type of the data.

```python
def data(self, index, role):
    if role == Qt.ItemDataRole.DisplayRole:
        # Get the raw value
        value = self._data[index.row()][index.column()]

        # Perform per-type checks and render accordingly.
        if isinstance(value, datetime):
            # Render time to YYY-MM-DD.
            return value.strftime("%Y-%m-%d")

        if isinstance(value, float):
            # Render float to 2 dp
            return "%.2f" % value

        if isinstance(value, str):
            # Render strings with quotes
            return '"%s"' % value

        # Default (anything not captured above: e.g. int)
        return value

```

In [2]:
from datetime import datetime


# model class subclass the QAbstractTableModel
# it has a 2D data structure with rows and columns
class TableModel(QtCore.QAbstractTableModel):
    """

    """
    def __init__(self, data):
        super(TableModel, self).__init__()
        self._data = data

    # return the data which view requires
    def data(self, index, role):
        # in this example, we only treat DisplayRole
        if role == Qt.ItemDataRole.DisplayRole:
           # Get the raw value
            value = self._data[index.row()][index.column()]

            # Perform per-type checks and render accordingly.
            if isinstance(value, datetime):
                # Render time to YYY-MM-DD.
                return value.strftime("%Y-%m-%d")

            if isinstance(value, float):
                # Render float to 2 dp
                return "%.2f" % value

            if isinstance(value, str):
                # Render strings with quotes
                return '"%s"' % value

            # Default (anything not captured above: e.g. int)
            return value

    # return the row count of current data store
    def rowCount(self, index):
        # The length of the outer list.
        return len(self._data)

    def columnCount(self, index):
        # The following takes the first sub-list, and returns
        # the length (only works if all rows are an equal length)
        return len(self._data[0])


class MainWindow(QtWidgets.QMainWindow):
    def __init__(self):
        super().__init__()

        # create a table view
        self.table = QtWidgets.QTableView()

        # use nested list as a 2-dimensional data store
        data = [
            [4, 9, 2],
            [1, -1, 'hello'],
            [3.023, 5, -5],
            [3, 3, datetime(2017,10,1)],
            [7.555, 8, 9],
        ]

        self.model = TableModel(data)
        self.table.setModel(self.model)

        self.setCentralWidget(self.table)


app=QtWidgets.QApplication(sys.argv)
window=MainWindow()
window.show()
app.exec()

qt.qpa.xcb: X server does not support XInput 2


0

## 3.3 Customize QTableView

Using colors and icons to highlight cells in data tables can help make data easier to find and understand, or help users to select or mark data of interest. Qt allows for complete control of all of these from the model, by responding to the relevant role on the data method.

Need to complete https://www.pythonguis.com/tutorials/pyqt6-qtableview-modelviews-numpy-pandas/

## 3.4 Use Pandas as data source

In previous example, we use a 2D list as our data source. Here we want to use a pandas dataframe as the data source

In [3]:
import pandas as pd

class TableModel(QtCore.QAbstractTableModel):

    def __init__(self, data):
        super(TableModel, self).__init__()
        self._data = data

    def data(self, index, role):
        if role == Qt.ItemDataRole.DisplayRole:
            value = self._data.iloc[index.row(), index.column()]
            return str(value)

    def rowCount(self, index):
        return self._data.shape[0]

    def columnCount(self, index):
        return self._data.shape[1]

    def headerData(self, section, orientation, role):
        # section is the index of the column/row.
        if role == Qt.ItemDataRole.DisplayRole:
            if orientation == Qt.Orientation.Horizontal:
                return str(self._data.columns[section])

            if orientation == Qt.Orientation.Vertical:
                return str(self._data.index[section])


class MainWindow(QtWidgets.QMainWindow):

    def __init__(self):
        super().__init__()

        self.table = QtWidgets.QTableView()

        data = pd.DataFrame([
          [1, 9, 2],
          [1, 0, -1],
          [3, 5, 2],
          [3, 3, 2],
          [5, 8, 9],
        ], columns = ['A', 'B', 'C'], index=['Row 1', 'Row 2', 'Row 3', 'Row 4', 'Row 5'])

        self.model = TableModel(data)
        self.table.setModel(self.model)

        self.setCentralWidget(self.table)


app=QtWidgets.QApplication(sys.argv)
window=MainWindow()
window.show()
app.exec()

0